In [7]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import math
from sklearn import preprocessing

In [8]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
  
def load_file(fileName, inputcols,outputcol):
    dataset = pd.read_table("kseindex.csv", header=0, sep=",", encoding="unicode_escape")
    data = dataset[inputcols]
    target = dataset[outputcol]

    trainingX,testX,trainingY,testY = train_test_split(data,target,test_size=0.33, random_state=43)

    # normalizing data
    trainingX = (trainingX - trainingX.min()) / (trainingX.max() - trainingX.min())
    trainingY = (trainingY -  trainingY.min()) / (trainingY.max() - trainingY.min())
    testX = (testX - testX.min()) / (testX.max() - testX.min())
    testY = (testY - testY.min()) / (testY.max() - testY.min())
  

  
    # # apply normalization techniques
    # IF WE USE df_max_scaled[column] = df_max_scaled[column] /df_max_scaled[column].abs().max() for normalization error
    # reduces to 0.02 from 0.08......
    
    return trainingX,testX,trainingY,testY

#print(trainingX.shape, testX.shape, trainingY.shape, testY.shape)

In [9]:
import math

def sigmoid(x):
    return(1/(1 + np.exp(-x)))

# def sigmoid_deriv(x):
#     return sigmoid(x) * (1 - sigmoid(x))

def backward_prop(a1,del_out,outputNeurons,hiddenNeurons,learningRate):
    #follows w = output- (learningRate*(Etol_weight) throught dot product (error in layers reduced)
    change = learningRate* (del_out).dot(np.resize(a1, (outputNeurons,hiddenNeurons)))
    # for the gradient
    change = np.transpose(np.resize(change,(outputNeurons,hiddenNeurons)))
    return change

def learn_model(trainingX, trainingY,inputNeurons, hiddenNeurons, outputNeurons, learningRate, epochs):
  
    classifier = {}
    weight_hid = np.random.rand(inputNeurons, hiddenNeurons)*0.01
    weight_out = np.random.rand(hiddenNeurons, outputNeurons)*0.01
    
    training_X = np.array(trainingX)
    # print(training_X) 
    training_Y = np.array(trainingY)
    # print(training_Y)

    # for j in range(epoch):
    for k in range(epochs):
        i = 0 #incremented later
        
        for x in training_X:
            # print(x)
            output_Hid_lst = []
            output_Out_lst= []
            errors_lst = []
            error_total = 0
        
            # Forward propagation
            # we are going to assign random weights to the input features.
            weights = np.random.randn(inputNeurons, hiddenNeurons) * 0.01
            # print(weights)    
            bias = np.random.randn((hiddenNeurons)) * 0.01
            # print(bias)
           
            # weight = np.dot(x, weights) + bias
            # for the hidden layer Forward propagationp.dot(output_Hidden, weights) + bias
            hidden_lay = np.dot(x, weight_hid) + bias
            # print(hidden_lay)
            a1 = sigmoid(hidden_lay)
            # print(a1)
            output_Hid_lst.append(a1)
            # print(output_Hidden)

            # for the output layer Forward propagation
            output_lay = np.dot(a1, weight_out)
            # print(output_lay)
            a2 = sigmoid(output_lay)
            # print(a2)
            output_Out_lst.append(a2)

            # Here we have the backward propagation
            # First we find the errors for the output layer
            # Etol_out = (target-out)* out(1-out) formulae later we dot with a1          
            Etol_out = (training_Y[i]-a2)
            out_net = a2*(1-a2)
            Etol_weight = out_net*Etol_out
            # print(Etol_weight)
            errors_lst.append(Etol_weight)
            # print(errors_lst)
        
            #for output layer (hidden to output weig change)
            Weights_o = backward_prop(a1,Etol_weight,outputNeurons,hiddenNeurons,learningRate)
            # print(Weights_o)
            weight_out = weight_out + Weights_o 
            # print(weight_out) 
            

            # Second we find the change for the hidden to output layer
            Error_hidden = np.dot(weight_out, Etol_weight)
            Etol_hidden = a1*(1-a1)*Error_hidden
            # print(Etol_hidden)
            Weights_h = backward_prop(x,Etol_hidden,hiddenNeurons,3,learningRate)
            # print(Weights_h)
            weight_hid = weight_hid + Weights_h
            # print(weight_hid) 
            i = i + 1
            
    classifier["hidden_arr"] = weight_hid 
    classifier["output_arr"] = weight_out
    
    # print(classifier)
    
    return classifier



In [10]:
def predict(classifier, testdata):
    predicted_val=[]
    #array for test data
    testx = np.array(testdata)
    #Your code to make predictions on test data using the learned model will go here
    for i in testx:
        hiddenNeurons = 4
        bias = np.random.randn((hiddenNeurons)) * 0.01
        hidden_lay = np.dot(i, classifier['hidden_arr']) + bias
        # print(hidden_lay)
        a1 = sigmoid(hidden_lay)
        # print(a1)
        
        # for the output layer Forward propagation
        output_lay = np.dot(a1, classifier['output_arr'])
        # print(output_lay)
        a2 = sigmoid(output_lay)
        # print(a2)
        predicted_val.append(a2[0])
        # print(predicted_val)
    return predicted_val

In [11]:
def computeError(actual_class, predicted_class):
        
    MSE = -1    
    #Your code to compute MSE will go here
    # for loss we will be using mean square error(MSE)

    s =(np.square(actual_class-predicted_class))
    MSE = np.sum(s)/len(predicted_class)
   
    
    
    return MSE

In [12]:
features, target = ["X1","X2","X3"],"Y"

print("Loading data.....")
trainingX,testX,trainingY,testY = load_file("kseindex.csv", features, target)

print("Learning model.....")
model = learn_model(trainingX,trainingY,3,4,1,0.8,10)

print("Classifying test data......")      
predictedY = predict(model, testX)

print("Evaluating results.....")
computeError(testY,predictedY)

Loading data.....
Learning model.....
Classifying test data......
Evaluating results.....


0.0867438978447767